In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from sklearn.model_selection import cross_val_score as cvs
from xgboost import XGBRegressor

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/financial-performance-prediction/sample_submission.csv
/kaggle/input/financial-performance-prediction/data_dictionary.txt
/kaggle/input/financial-performance-prediction/train.csv
/kaggle/input/financial-performance-prediction/test.csv


In [2]:
train = pd.read_csv('/kaggle/input/financial-performance-prediction/train.csv')
test = pd.read_csv('/kaggle/input/financial-performance-prediction/test.csv')

In [3]:
print(f'Train shape: {train.shape}')
print(f'Test shape: {test.shape}')

Train shape: (1624, 212)
Test shape: (406, 203)


In [4]:
targets_names = [col for col in train if col not in test]
print(f'Target columns: {targets_names}')

Target columns: ['Q0_TOTAL_ASSETS', 'Q0_TOTAL_LIABILITIES', 'Q0_TOTAL_STOCKHOLDERS_EQUITY', 'Q0_GROSS_PROFIT', 'Q0_COST_OF_REVENUES', 'Q0_REVENUES', 'Q0_OPERATING_INCOME', 'Q0_OPERATING_EXPENSES', 'Q0_EBITDA']


In [5]:
train.dtypes.value_counts()

float64    207
object       4
int64        1
Name: count, dtype: int64

In [6]:
test.dtypes.value_counts()

float64    198
object       4
int64        1
Name: count, dtype: int64

In [7]:
# make an easy col to split back the train and test sets
train['is_train'] = True
test['is_train'] = False
df = pd.concat([train, test], axis = 0, ignore_index = True)

In [8]:
df.select_dtypes(include = 'O')

,industry,sector,recommendationKey,financialCurrency
0,Personal Services,Consumer Cyclical,strong_buy,USD
1,Building Products & Equipment,Industrials,buy,USD
2,NaN,Unknown,NaN,NaN
3,Scientific & Technical Instruments,Technology,hold,USD
4,Drug Manufacturers - Specialty & Generic,Healthcare,buy,USD
...,...,...,...,...
2025,Packaged Foods,Consumer Defensive,hold,USD
2026,NaN,Unknown,NaN,NaN
2027,Medical Devices,Healthcare,buy,USD
2028,Drug Manufacturers - Specialty & Generic,Healthcare,buy,USD


In [9]:
for col in df.select_dtypes(include = 'O'):
    print(f'{col} unique values num: {df[col].nunique()}')
    print('-'*50)

industry unique values num: 113
--------------------------------------------------
sector unique values num: 10
--------------------------------------------------
recommendationKey unique values num: 6
--------------------------------------------------
financialCurrency unique values num: 2
--------------------------------------------------


In [10]:
def binarize(df, col):
    uniques = list(df[col].dropna().unique())
    df[col] = [1 if val == uniques[0] else 0 for val in df[col]]
    return df

In [11]:
def one_hot_encode(df, col):
    dummies = pd.get_dummies(df[col], prefix = col).astype(int)
    df.drop(col, axis = 1, inplace = True)
    df = pd.concat([df, dummies], axis = 1)
    return df

In [12]:
def factorize(df, col):
    mapping = {name: ix for ix, name in enumerate(df[col].unique())}
    df[col] = df[col].map(mapping)
    return df

In [13]:
df = binarize(df, 'financialCurrency')
print(df['financialCurrency'].head())

0    1
1    1
2    0
3    1
4    1
Name: financialCurrency, dtype: int64


In [14]:
print(df.shape)
df = one_hot_encode(df, 'sector')
df = one_hot_encode(df, 'recommendationKey')
print(df.shape)

(2030, 213)
(2030, 227)


In [15]:
df = factorize(df, 'industry')
print(df.dtypes.value_counts())

float64    207
int64       19
bool         1
Name: count, dtype: int64


In [16]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [17]:
df.isnull().sum()

Id                                  0
industry                            0
fullTimeEmployees                 102
auditRisk                         625
boardRisk                         625
                                 ... 
recommendationKey_hold              0
recommendationKey_none              0
recommendationKey_sell              0
recommendationKey_strong_buy        0
recommendationKey_underperform      0
Length: 227, dtype: int64

In [18]:
print('Missing data stats:\n')
under_10_percent = 0
ten_to_20_percent = 0
twenty_to_50_percent = 0
over_50_percent = 0
for col in df:
    if np.any(df[col].isnull()):
        percent_missing = np.round(100*df[col].isnull().sum()/len(df),2)
        if percent_missing < 10:
            under_10_percent+=1
        elif 10 <= percent_missing < 20:
            ten_to_20_percent+=1
        elif 20 <= percent_missing < 50:
            twenty_to_50_percent+=1
        else:
            over_50_percent+=1
print(f'ncols with under 10% missing data:\t{under_10_percent}')
print(f'ncols with 10-20% missing data:\t\t{ten_to_20_percent}')
print(f'ncols with 20-50% missing data:\t\t{twenty_to_50_percent}')
print(f'ncols with over 50% missing data:\t{over_50_percent}')

Missing data stats:

ncols with under 10% missing data:	134
ncols with 10-20% missing data:		44
ncols with 20-50% missing data:		16
ncols with over 50% missing data:	1


In [19]:
# check if there are any inf values in data
df[df == np.inf].count().sum()

17

In [20]:
# check for missing values in the target columns
train[targets_names].isnull().sum()

Q0_TOTAL_ASSETS                 0
Q0_TOTAL_LIABILITIES            0
Q0_TOTAL_STOCKHOLDERS_EQUITY    0
Q0_GROSS_PROFIT                 0
Q0_COST_OF_REVENUES             0
Q0_REVENUES                     0
Q0_OPERATING_INCOME             0
Q0_OPERATING_EXPENSES           0
Q0_EBITDA                       0
dtype: int64

In [21]:
def impute_missing_by_median(df):
    print(f'Missing values before imputation: {sum(df.isnull().sum())}')
    # first replace inf and -inf with nan
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    # then impute nan by median
    for col in df:
        if np.any(df[col].isnull()):
            df[col].fillna(df[col].median(), inplace = True)
    print(f'Missing values after imputation: {sum(df.isnull().sum())}')
    return df

In [22]:
df = impute_missing_by_median(df)

Missing values before imputation: 28653
Missing values after imputation: 0


In [23]:
train = df[df['is_train'] == True]
test = df[df['is_train'] == False]
train.drop('is_train', axis = 1, inplace=True)
test.drop('is_train', axis = 1, inplace=True)
test.reset_index(drop = True, inplace = True)

test.drop(targets_names, axis = 1, inplace = True)
targets = train[targets_names].copy()
train.drop(targets_names, axis = 1, inplace = True)

/tmp/ipykernel_18/4229629428.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.drop('is_train', axis = 1, inplace=True)
/tmp/ipykernel_18/4229629428.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.drop('is_train', axis = 1, inplace=True)
/tmp/ipykernel_18/4229629428.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.drop(targets_names, axis = 1, inplace = True)
/tmp/ipykernel_18/4229629428.py:9: SettingWithCopyWa

In [24]:
targets

,Q0_TOTAL_ASSETS,Q0_TOTAL_LIABILITIES,Q0_TOTAL_STOCKHOLDERS_EQUITY,Q0_GROSS_PROFIT,Q0_COST_OF_REVENUES,Q0_REVENUES,Q0_OPERATING_INCOME,Q0_OPERATING_EXPENSES,Q0_EBITDA
0,1.192950e+09,1.055816e+09,1.371340e+08,25598000.0,6.189900e+07,4.299200e+07,1.522200e+07,10383000.0,15215000.0
1,1.219954e+09,8.350460e+08,3.849080e+08,111786000.0,2.177960e+08,3.295820e+08,2.890800e+07,85522000.0,26264000.0
2,-1.358200e+11,-3.797200e+10,-9.784800e+10,-435000000.0,4.445000e+09,4.010000e+09,-1.472000e+09,219000000.0,-654000000.0
3,8.329800e+07,1.604000e+07,-3.163300e+07,494000.0,3.339000e+06,3.833000e+06,-1.234800e+07,12842000.0,-12348000.0
4,1.485460e+08,5.655500e+07,9.199100e+07,1761000.0,-2.375000e+06,-6.140000e+05,-1.009800e+07,11859000.0,-10098000.0
...,...,...,...,...,...,...,...,...,...
1619,3.539600e+08,1.997590e+08,1.542010e+08,48741000.0,4.569300e+07,9.443400e+07,2.616000e+06,46125000.0,2616000.0
1620,1.061530e+08,3.949300e+07,6.811700e+07,1077000.0,1.690000e+05,1.246000e+06,-6.705000e+06,7782000.0,-6705000.0
1621,5.203150e+08,3.807960e+08,3.233080e+08,27808000.0,1.123840e+08,1.401920e+08,7.061000e+06,9417000.0,18391000.0
1622,1.997210e+08,2.692100e+07,1.728000e+08,0.0,0.000000e+00,0.000000e+00,-3.391900e+07,33919000.0,-33919000.0


In [25]:
model = XGBRegressor()

In [26]:
print('Cross validation R2 scores for each target:\n')
cross_val_score_results = {}
for target in targets:
    score = np.round(np.mean(cvs(model, train, targets[target], cv=3, scoring='r2')),2)
    cross_val_score_results[target] = score
    print(f'{target} -> {score}')
print(f'\nMean R2 score across all targets: {np.mean(list(cross_val_score_results.values()))}')

Cross validation R2 scores for each target:

Q0_TOTAL_ASSETS -> 0.62
Q0_TOTAL_LIABILITIES -> 0.4
Q0_TOTAL_STOCKHOLDERS_EQUITY -> 0.87
Q0_GROSS_PROFIT -> 0.73
Q0_COST_OF_REVENUES -> 0.83
Q0_REVENUES -> 0.9
Q0_OPERATING_INCOME -> 0.72
Q0_OPERATING_EXPENSES -> 0.76
Q0_EBITDA -> -0.42

Mean R2 score across all targets: 0.6011111111111112


In [27]:
preds = {}
for target in targets:
    model.fit(train, targets[target])
    pred = model.predict(test)
    preds[target] = pred
    print(f'Finished train/predict for: {target}')

Finished train/predict for: Q0_TOTAL_ASSETS
Finished train/predict for: Q0_TOTAL_LIABILITIES
Finished train/predict for: Q0_TOTAL_STOCKHOLDERS_EQUITY
Finished train/predict for: Q0_GROSS_PROFIT
Finished train/predict for: Q0_COST_OF_REVENUES
Finished train/predict for: Q0_REVENUES
Finished train/predict for: Q0_OPERATING_INCOME
Finished train/predict for: Q0_OPERATING_EXPENSES
Finished train/predict for: Q0_EBITDA


In [28]:
sub = pd.read_csv('/kaggle/input/financial-performance-prediction/sample_submission.csv')

In [29]:
# sanity check
sub.Id == test.Id

0      True
1      True
2      True
3      True
4      True
       ... 
401    True
402    True
403    True
404    True
405    True
Name: Id, Length: 406, dtype: bool

In [30]:
for target in preds:
    sub[target] = preds[target]
sub.head()

,Id,Q0_REVENUES,Q0_COST_OF_REVENUES,Q0_GROSS_PROFIT,Q0_OPERATING_EXPENSES,Q0_EBITDA,Q0_OPERATING_INCOME,Q0_TOTAL_ASSETS,Q0_TOTAL_LIABILITIES,Q0_TOTAL_STOCKHOLDERS_EQUITY
0,480,-1.211120e+06,-3.895045e+05,-5.017264e+04,-7.105477e+05,-1.565854e+06,-1.995759e+06,-2.966194e+07,-1.779459e+06,4.574116e+05
1,573,5.949692e+08,3.356995e+08,1.815985e+08,1.065167e+08,1.317824e+08,4.421544e+07,3.533268e+09,1.871350e+09,5.346400e+08
2,1967,-1.272589e+06,1.030028e+07,-6.526061e+05,4.120479e+06,-6.113872e+06,-5.567920e+06,5.888865e+07,2.424615e+07,1.971546e+06
3,57,4.606910e+08,2.904094e+08,1.891734e+08,7.999364e+07,6.343608e+07,6.295966e+07,6.022530e+09,2.713045e+09,3.356197e+09
4,131,9.713140e+08,8.411632e+08,1.933180e+08,8.722598e+07,1.275100e+08,1.266720e+08,4.397583e+09,2.879660e+09,1.119394e+09


In [31]:
sub.to_csv('submission.csv', index = False)